БИБЛИОТЕКИ

In [1]:
import numpy as np
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import recall_score
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

ЧТЕНИЕ МАССИВОВ

In [2]:
# монтируем свой диск...
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ССЫЛКИ НА ФАЙЛЫ ДЛЯ ШАГА 2 - ОТСЕЧЕНИЕ КЛАССА '0010'

In [3]:
# входные данные
x_train_file = '/content/drive/My Drive/Bases/DTP/x_train_1.dat'
y_train_file = '/content/drive/My Drive/Bases/DTP/y_train_1.dat'

# выходный файл с весами лучшей модели
model_save_file = '/content/drive/My Drive/Bases/DTP/model_1.h5'

ЧТЕНИЕ ДАННЫХ

In [4]:
# чтение
x_train = np.fromfile(x_train_file)
x_train = x_train.reshape((4778, 7))
y_train = np.fromfile(y_train_file)

# 5% на проверочную выборку
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.05)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(4539, 7)
(4539,)
(239, 7)
(239,)


НАСТРОЙКА ВЕСОВ БИНАРНОЙ КЛАССИФИКАЦИИ

In [5]:
# веса модели
weight_for_0 = 1.0 / (y_train == 0.0).sum()
weight_for_1 = 1.0 / (y_train == 1.0).sum()

class_weight = {0: weight_for_0, 1: weight_for_1}

print(class_weight)

{0: 0.00036023054755043225, 1: 0.0005672149744753262}


МОДЕЛЬ НЕЙРОННОЙ СЕТИ

In [6]:
def CreateModel(activation='linear', dropout=0.1):

  input = Input((7,)) 

  x = BatchNormalization()(input)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)
  
  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  output = Dense(1, activation='sigmoid')(x) 
  
  return Model(input, output) 

CALLBACK, сохраняющий лучший результат по метрике Recall

In [7]:
class MyCallback(Callback):
  
  # инициализация
  def __init__(self, model, x_val, y_val):
    # модель
    self.model = model 
    # проверочная выборка
    self.x_val = x_val 
    self.y_val = y_val
    # начальный счет по метрике
    self.score = 0
       
  # подсчет метрики Recall для val  
  def calc_recall(self):
    y_pred = self.model.predict(self.x_val)
    y_p = np.around(y_pred).astype(int)
    y_t = np.around(self.y_val).astype(int)
    return recall_score(y_t, y_p, average='macro')

  # окончение каждой эпохи....
  def on_epoch_end(self, epoch, logs=None):
    score_0 = self.calc_recall()
    print('\n\nТекущий Recall =', score_0, '\n')
    if score_0 > self.score:
      self.score = score_0
      print('\n\n!!!!!!!!Лучший Recall =', score_0, '\n')
      self.model.save_weights(model_save_file)   

  # в конце обучения
  def on_train_end(self, logs=None):
    print('\n\n!ЛУЧШИЙ Recall ОБУЧЕНИЯ =', self.score, '\n')


In [39]:
# параметры обучения

batch_size = 10

dropout = 0.5

activate_function = 'relu'

epochs_count = 200

learning_rate = 1e-4

ОБУЧЕНИЕ МОДЕЛИ БИНАРНОЙ КЛАССИФИКАЦИИ ШАГ 1

In [40]:
del model

In [41]:
# создание модели
model = CreateModel(activation=activate_function, dropout=dropout)  
cb = MyCallback(model, x_val, y_val)
#cb = MyCallback(model, x_train, y_train)

In [42]:
# компиляция
model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss=['binary_crossentropy'], 
              metrics=['accuracy'])

# обучение
history = model.fit(x_train, y_train, 
                    batch_size = batch_size,
                    epochs= epochs_count, 
                    validation_data=(x_val, y_val), 
                    verbose=1, 
                    class_weight = class_weight,
                    callbacks=[cb])

Epoch 1/200
443/454 [============================>.] - ETA: 0s - loss: 3.7512e-04 - accuracy: 0.4953

Текущий Recall = 0.5071148540728909 



!!!!!!!!Лучший Recall = 0.5071148540728909 

454/454 [==============================] - 3s 5ms/step - loss: 3.7349e-04 - accuracy: 0.4972 - val_loss: 0.6914 - val_accuracy: 0.5230
Epoch 2/200
445/454 [============================>.] - ETA: 0s - loss: 3.6819e-04 - accuracy: 0.4944

Текущий Recall = 0.5288587193262668 



!!!!!!!!Лучший Recall = 0.5288587193262668 

454/454 [==============================] - 2s 4ms/step - loss: 3.6871e-04 - accuracy: 0.4946 - val_loss: 0.6925 - val_accuracy: 0.5314
Epoch 3/200
443/454 [============================>.] - ETA: 0s - loss: 3.6165e-04 - accuracy: 0.4912

Текущий Recall = 0.48159576012777694 

454/454 [==============================] - 2s 4ms/step - loss: 3.6100e-04 - accuracy: 0.4911 - val_loss: 0.6918 - val_accuracy: 0.5063
Epoch 4/200
444/454 [============================>.] - ETA: 0s - loss: 3.4218e-0

ПРОВЕРИМ RECALL

In [43]:
# чтение весов
model.load_weights('/content/drive/My Drive/Bases/DTP/model_1.h5')

In [44]:
y_pred = model.predict(x_val)
y_p = np.around(y_pred).astype(int)
y_t = np.around(y_val).astype(int)

print(recall_score(y_t, y_p, average='macro'))

0.5606940612748657
